### 라이브러리 로드

In [1]:
from elasticsearch import Elasticsearch, helpers
import pandas as pd
import csv
from pprint import pprint # Json 출력 인덴트 주어 보기 편하게 출력.
import pandas as pd # 데이터프레임 가공을 위한 라이브러리.
import psycopg2 # 데이터베이스 연동을 위한 라이브러리.
from konlpy.tag import Twitter, Kkma # 한글 형태소 분석 위한 라이브러리.

### DB 불러오기

In [2]:
db = psycopg2.connect(host='192.168.1.222', dbname='omop',user='belab',password='qwer1234',port=5433) # 파이썬 db 연동.

### Table 불러오기

In [3]:
pheno = pd.read_sql("""select tt.* from
(
select 'pheno' as gubun, pheno_id as id, '' as naire_name, '' as pheno_name, '' as qualifier_name, pheno as value from munjin.pheno2 
where category = 'clinical finding' and pheno_id > 130 
union all
select 'munjin' as gubun, munjin_id as id ,naire_name, pheno_name, qualifier_name, value from munjin.munjin2 
where qualifier_name in ('Character', 'Aggravated by', 'Provoked by', 'Risk factor', 'Symptom relieving factors')) as tt
order by value collate "C";""", db)

### Twitter api 사용, 증현상/설명값 품사 분석

In [11]:
twitter = Twitter()
pheno['morpheme_list']=pheno['value'].apply(twitter.pos)

In [40]:
for i in range(1,30):
    print(pheno['morpheme_list'][i])

[]
[]
[]
[]
[]
[]
[]
[]
[('1', 'Number'), (',', 'Punctuation'), ('2', 'Number'), (',', 'Punctuation'), ('3', 'Number'), (',', 'Punctuation'), ('4', 'Number'), ('번째', 'Suffix'), ('손가락', 'Noun'), ('을', 'Josa'), ('포함', 'Noun'), ('한', 'Josa'), ('부위', 'Noun'), ('가', 'Josa'), ('움직이기', 'Verb'), ('어려움', 'Noun')]
[('4', 'Number'), (',', 'Punctuation'), ('5', 'Number'), ('번째', 'Suffix'), ('손가락', 'Noun'), ('부위', 'Noun'), ('가', 'Josa'), ('움직이기', 'Verb'), ('어려움', 'Noun')]
[('4', 'Number'), ('번', 'Noun'), ('중', 'Noun'), ('1', 'Number'), ('번', 'Noun'), ('변', 'Noun'), ('이', 'Josa'), ('남아있는', 'Verb'), ('느낌', 'Noun')]
[('4', 'Number'), ('번', 'Noun'), ('중', 'Noun'), ('1', 'Number'), ('번', 'Noun'), ('손가락', 'Noun'), ('을', 'Josa'), ('써야', 'Verb'), ('함', 'Noun')]
[('4', 'Number'), ('번', 'Noun'), ('중', 'Noun'), ('1', 'Number'), ('번', 'Noun'), ('항문', 'Noun'), ('이', 'Josa'), ('막히는', 'Verb'), ('느낌', 'Noun')]
[('4', 'Number'), ('번', 'Noun'), ('중', 'Noun'), ('1', 'Number'), ('번', 'Noun'), ('힘', 'Noun'), ('을', 'Jos

### 명사에 해당하는 형태소만 추출

In [47]:
Noun_list=[]
Verb_list=[]
Adjective_list=[]
Adverb_list=[]
Conjunction_list=[]
Number_list=[]


for i in pheno['morpheme_list'] :
    a = []
    b = []
    c = []
    d = []
    e = []
    f = []
    
    for j in i :
        term = j[0]
        pos = j[-1]
        if pos == 'Noun' or pos == 'Foreign' or pos == 'Alpha' :
            a.append(term)
        elif pos == 'Verb' :
            b.append(term)
        elif pos == 'Adjective' :
            c.append(term)
        elif pos == 'Adverb' :
            d.append(term)
        elif pos == 'Conjunction' :
            e.append(term)
        elif pos == 'Number' :
            f.append(term)
            
    Noun_list.append(a)
    Verb_list.append(b)
    Adjective_list.append(c)
    Adverb_list.append(d)
    Conjunction_list.append(e)
    Number_list.append(f)
    

pheno['noun_list'] = Noun_list
pheno['verb_list'] = Verb_list
pheno['adjective_list'] = Adjective_list
pheno['adverb_list'] = Adverb_list
pheno['conjunction_list'] = Conjunction_list
pheno['number_list'] = Number_list

### CSV 저장

In [48]:
pheno.to_csv('211014_명사_추출본.csv', encoding='euc-kr')